In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

PROJECT_ID = "tese-463714"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
  PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_REGION"] = LOCATION

In [2]:
MODEL_ID = "gemini-2.0-flash"

api_host = "aiplatform.googleapis.com"
if LOCATION != "global":
  api_host = f"{LOCATION}-aiplatform.googleapis.com"

os.environ["API_ENDPOINT"] = (
    f"{api_host}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/publishers/google/models/{MODEL_ID}"
)
API_ENDPOINT = os.environ["API_ENDPOINT"]

API_KEY = os.environ.get("GOOGLE_API_KEY")

In [3]:
# Updated third cell from your notebook

import os
from pyrit.memory.central_memory import CentralMemory
from pyrit.prompt_target import HTTPTarget
import json
from urllib.parse import urlparse
from pyrit.common import IN_MEMORY, initialize_pyrit

# Import the google-auth library
import google.auth
import google.auth.transport.requests

initialize_pyrit(memory_db_type=IN_MEMORY)
memory = CentralMemory.get_memory_instance()

# --- NEW: Function to get an OAuth2 Access Token ---
# --- CORRECTED: Function to get an OAuth2 Access Token with the right scope ---
def get_access_token():
    """Generates an OAuth2 access token from a service account key."""
    
    # 1. Load the credentials from the file
    credentials, project = google.auth.load_credentials_from_file("tese-463714-8b2283314b9a.json")
    
    # 2. Add the necessary scope to the credentials
    scoped_credentials = credentials.with_scopes(['https://www.googleapis.com/auth/cloud-platform'])
    
    # 3. Refresh the credentials to get a token
    auth_req = google.auth.transport.requests.Request()
    scoped_credentials.refresh(auth_req)
    
    # 4. Return the token
    return scoped_credentials.token

# --- Your existing callback function ---
def gemini_response_callback(response) -> str:
    """Extract response from Gemini API response"""
    try:
        response_json = json.loads(response.content)
        return response_json["candidates"][0]["content"]["parts"][0]["text"]
    except (KeyError, IndexError, TypeError, json.JSONDecodeError):
        return str(response.content)

# --- MODIFIED: The HTTP request now uses the access token ---

# Generate the token
access_token = get_access_token()

# Parse the API endpoint to get host and path
parsed_url = urlparse(f"https://{API_ENDPOINT}:generateContent")
host = parsed_url.netloc
path = parsed_url.path

# Create raw HTTP request string, now using the real token
raw_http_request = f"""POST {path} HTTP/1.1
Host: {host}
Content-Type: application/json
Authorization: Bearer {access_token}

{{
  "contents": {{
    "role": "USER",
    "parts": {{ "text": "{{{{PROMPT}}}}" }}
  }},
  "generation_config": {{
    "response_modalities": "TEXT"
  }}
}}"""

Gemini_target = HTTPTarget(
    http_request=raw_http_request,
    callback_function=gemini_response_callback,
    timeout=None
)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
import pathlib

from pyrit.models.seed_prompt import SeedPromptDataset
from pyrit.prompt_normalizer.normalizer_request import NormalizerRequest


malicious_or_vulnerable = "or_bench"


if malicious_or_vulnerable == "or_bench":

    or_bench_dataset= SeedPromptDataset.from_yaml_file(pathlib.Path("datasets/or-bench-hard-1k.yaml"))
    label = "or_bench"
    await memory.add_seed_prompts_to_memory_async(prompts=or_bench_dataset.prompts, added_by="jd")  # type: ignore
    prompt_groups = memory.get_seed_prompt_groups()
    requests = [NormalizerRequest(seed_prompt_group=p) for p in prompt_groups]

else:
    with open(f"datasets/{malicious_or_vulnerable}_goals.json", "r") as f:
        goals_list = json.load(f)

    goals_list = [goal['Prompt'] for goal in goals_list]

    label = [f"{malicious_or_vulnerable}_goals"]

    del malicious_or_vulnerable


In [5]:
import os
import pathlib
from dotenv import load_dotenv

from pyrit.prompt_target.ollama_chat_target import OllamaChatTarget
from pyrit.score.self_ask_refusal_scorer import SelfAskRefusalScorer

load_dotenv()

ollama_host = os.getenv("OLLAMA_HOST")

seed=444
temperature_judges=0.1

memory_labels = {"op_name": label, "user_name": "jd"}


jury_models = ["deepseek-r1:70b", "qwen2.5:latest", "dolphin3:8b"]

jury_1 = OllamaChatTarget(model_name=jury_models[0], endpoint=ollama_host + "/api/chat", timeout=None, options={'num_predict': 5000, 'seed': seed, 'temperature': temperature_judges})
jury_2 = OllamaChatTarget(model_name=jury_models[1], endpoint=ollama_host + "/api/chat", timeout=None, options={'num_predict': 1000, 'seed': seed, 'temperature': temperature_judges})
jury_3 = OllamaChatTarget(model_name=jury_models[2], endpoint=ollama_host + "/api/chat", timeout=None, options={'num_predict': 1000, 'seed': seed, 'temperature': temperature_judges})

refusal_scorer_jury_1 = SelfAskRefusalScorer(
    chat_target=jury_1,
    system_prompt_path=pathlib.Path("pyrit/datasets/score/refusal/refusal_to_generate_code.yaml")
)

refusal_scorer_jury_2 = SelfAskRefusalScorer(
    chat_target=jury_2,
    system_prompt_path=pathlib.Path("pyrit/datasets/score/refusal/refusal_to_generate_code.yaml")
)

refusal_scorer_jury_3 = SelfAskRefusalScorer(
    chat_target=jury_3,
    system_prompt_path=pathlib.Path("pyrit/datasets/score/refusal/refusal_to_generate_code.yaml")
)

In [6]:
# Test the Gemini target
import uuid
from pyrit.models import PromptRequestPiece, PromptRequestResponse
from pyrit.orchestrator.single_turn.prompt_sending_orchestrator import PromptSendingOrchestrator
from pyrit.prompt_normalizer.normalizer_request import NormalizerRequest


prompt_sending_orchestrator=  PromptSendingOrchestrator(Gemini_target, scorers=[refusal_scorer_jury_1, refusal_scorer_jury_2, refusal_scorer_jury_3], verbose=True, batch_size=3)

if malicious_or_vulnerable == "or_bench":
    requests = requests[1000:1250]

else: 
    requests: list[NormalizerRequest] = []
    for prompt in goals_list:
        requests.append(
            prompt_sending_orchestrator._create_normalizer_request(
            prompt_text=prompt,
            prompt_type="text",
            converters=prompt_sending_orchestrator._prompt_converters,
            #metadata={"task": goals_dictionary[prompt]},
            conversation_id=str(uuid.uuid4()),
    )
    )



responses = await prompt_sending_orchestrator.send_normalizer_requests_async(prompt_request_list=requests, memory_labels=memory_labels) #type: ignore
await prompt_sending_orchestrator.print_conversations_async()  # type: ignore


INFO:pyrit.memory.central_memory:Using existing memory instance: DuckDBMemory
INFO:httpx:HTTP Request: POST https://aiplatform.googleapis.com/v1/projects/tese-463714/locations/global/publishers/google/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aiplatform.googleapis.com/v1/projects/tese-463714/locations/global/publishers/google/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aiplatform.googleapis.com/v1/projects/tese-463714/locations/global/publishers/google/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aiplatform.googleapis.com/v1/projects/tese-463714/locations/global/publishers/google/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://aiplatform.googleapis.com/v1/projects/tese-463714/locations/global/publishers/google/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
INFO:httpx:HTTP R

Exception: Error scoring prompt with original prompt ID: 43b0f13d-bf41-4272-a73e-517028ceb2ff

In [7]:
prompt_sending_orchestrator.output_conversations_to_json(file_path="gemini_2.0_flash_or_bench_5th_250.json")